<a href="https://colab.research.google.com/github/zulqurtubia/KecerdasanBuatan/blob/main/Quiz_Kecerdasan_Buatan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import math 
import random

#Soal
#Carilah Nilai a, b, c, d dari persamaan a+2b+3c+4d = 30
#Membuat algoritma genetika untuk mencari masing-masing nilai dari a, b, c, dan d

class Algoritma_genetika :

	jum_chrom = 6 #Jumlah chromosome 6 populasi
	gen = ["a", "b", "c", "d"] #nilai awal gen a, b, c, dan d
	nilai_per_gen = {
		'min' : 0,
		'max' : 10
	} #nilai untuk setiap gen adalah antara 0-10
    
	crossover_rate = 50/100 #persen
	mutasi_rate = 10/100 #persen
	total_gen = 24 # total_gen didapatkan dari jumlah gen dalam chromose * jumlah populasi (4*6=24)
	next_gen = np.arange(4)
	stop = False

#Melakukan inisialisasi chromosome secara random    
	def __init__(self):
		self.first_chrom = np.random.randint(low=self.nilai_per_gen['min'], high=self.nilai_per_gen['max'], size=(self.jum_chrom, len(self.gen)))
		print(self.first_chrom)
		print("================================================")

#Melakukan evaluasi chromosome (menghitung fungsi objektif dari chromosome)
#Mencari fungsi objektif
#fungsi_objektif(chromosome) = | (a+2b+3c+4d) – 30 |
	def evaluasi_chrom(self, chrom, generasi):
		print("Generasi ["+str(generasi)+"] ------")
		
		jum_chromo = len(chrom)
		o = np.arange(jum_chromo)   
		fitness = np.arange(jum_chromo, dtype='f')
		
		#FUNGSI OBJEKTIF 
		#fo=fungsi_objektif(chromosome[x])=o[x]
		for x in range(len(chrom)):
			fo = abs((chrom[x][0]+2*chrom[x][1]+3*chrom[x][2]+4*chrom[x][3])-30)
			o[x] = fo
			#Melakukan selesksi Chromose dan mencari Fitnes
			fitn = 1/(fo+1)
			
			fitness[x] = fitn
			if(fitn == 1):
				self.stop = True
			print("Chromosome {0} : {1}, fitness = {2}".format(x, np.array2string(chrom[x], separator=','), fitn))
		print("Fitness Selesai")
		print(o)
		
        
#Mencari probabilitas
		Probab = np.arange(jum_chromo, dtype='f')
		total_fitness = fitness.sum() #rumus untuk total fitness
		Probab = fitness / total_fitness #rumus dalam mencari probabilitas
		print("Total Fitness : {}".format(str(total_fitness)))
		print("Rata-rata Fitness : {}".format(str(np.average(fitness))))
		print("Probabilitas : {}".format(np.array2string(Probab, separator=',')))
		print("Probabilitas yang Paling Tinggi : {}, pada chromosome ke {}".format(Probab[Probab.argmax()], str(Probab.argmax())))
		print("Chromosome yang mungkin terpilih : {}".format(np.array2string(chrom[Probab.argmax()], separator=',')))
		print("")
		print("")
		self.next_gen = chrom[Probab.argmax()]

#Melakukan seleksi dengan menggunakan roulete wheel (C) dari cumulative probabilitas
		C = np.arange(jum_chromo, dtype='f')
		total_x = 0
		for x in range(len(Probab)):
			total_x += Probab[x]
			C[x] = total_x

#roulete whell akan diputar sebanyak jumlah populasi (dalam kasus ini 6) secara acak
		R = np.random.sample(len(fitness))
		chrom_baru = np.arange(jum_chromo*len(self.gen)).reshape(jum_chromo, len(self.gen))

#Mencari chromosome baru berdasarkan roulete wheel yang telah dilakukan
		for y in range(len(R)):
			for k in range(len(chrom_baru)):
				if(R[y] < C[0]):
					chrom_baru[y] = chrom[0]
				elif((C[k-1] < R[y]) & (R[y] < C[k])):
					chrom_baru[y] = chrom[k]

#Pencarian crossover chromosome
		R = np.random.sample(jum_chromo)
		index_chrom_parent = [] # [1,2,3, ...]
		for p in range(len(R)):
			if(R[p] < self.crossover_rate):
				index_chrom_parent.append(p)
                
#Menentukan posisi crossover
		posisi_cros = np.random.randint(low=1, high=len(self.gen), size=len(index_chrom_parent)) #membangkitkan bilangan acak dengan batasan 1 sampai (panjang chromosome-1), dalam kasus ini bilangan acak yang dibangkitkan adalah 1 – 3.

#Menentukan posisi cut-point crossover
		offspring = np.arange(len(self.gen)*len(index_chrom_parent)).reshape(len(index_chrom_parent), len(self.gen))
		for i_parent in range(len(index_chrom_parent)):
			index_chrome_1 = index_chrom_parent[i_parent]
			if(i_parent == len(index_chrom_parent)-1):
				index_chrome_2 = index_chrom_parent[0]
			else:
				index_chrome_2 = index_chrom_parent[i_parent+1]
			#Melakukan cut-point
			cut_point = posisi_cros[i_parent]
			for p in range(len(chrom_baru[index_chrome_1])):
				#Loopin dilakukan berdasarkan generasi
				if(p >= posisi_cros[i_parent]):
					#Apabila generasi [p] >= bilangan acak[p], Mmaka akan diganti dengan generasi ke-2
					offspring[i_parent][p] = chrom_baru[index_chrome_2][p]
				else:
					offspring[i_parent][p] = chrom_baru[index_chrome_1][p]
    
		
		#Crossover digabung dengan chromosome baru
		for x in range(len(offspring)):
			chrom_baru[index_chrom_parent[x]] = offspring[x] #Penyimpanan crossover disimpa dalam variable offspring

		total_gen = len(chrom) * len(chrom[0])
		jum_mutasi = self.mutasi_rate * total_gen #rumus menghitung jumlah mutasi
		jum_mutasi = int(jum_mutasi)

		random_i_mutasi = np.random.randint(low=0, high=total_gen, size=jum_mutasi)

		for x in range(len(random_i_mutasi)):
			index_mutasi = random_i_mutasi[x]
			byk_kromosom = len(chrom)
			byk_gen = len(chrom[0])
			random_value = random.randint(self.nilai_per_gen['min'], self.nilai_per_gen['max'])
			if(index_mutasi <= byk_gen):#apabila index_mutasi <= banyak gen maka gen pada chromosome baru yang ke-0 akan diganti
				chrom_baru[0][index_mutasi-1]
			else:
				#Mencari index dari posisi Y chromosome
				posisi_y = index_mutasi/byk_gen
				posisi_y = int(posisi_y)
				posisi_x = index_mutasi % byk_gen
				chrom_baru[posisi_y][posisi_x] = random_value
		return chrom_baru

	def do_now(self):
		chromosome_current = self.first_chrom
		for generasi in range(0, self.total_gen):
			if(self.stop != True):
				chromosome_current = self.evaluasi_chrom(chromosome_current, generasi)
		print("================================================")
		print("Chromosome Terbaik Adalah:")
		print(self.next_gen)

run = Algoritma_genetika()
run.do_now()

[[6 9 1 9]
 [5 2 9 3]
 [2 7 9 0]
 [1 8 2 4]
 [9 1 0 1]
 [7 3 7 5]]
Generasi [0] ------
Chromosome 0 : [6,9,1,9], fitness = 0.029411764705882353
Chromosome 1 : [5,2,9,3], fitness = 0.05263157894736842
Chromosome 2 : [2,7,9,0], fitness = 0.07142857142857142
Chromosome 3 : [1,8,2,4], fitness = 0.1
Chromosome 4 : [9,1,0,1], fitness = 0.0625
Chromosome 5 : [7,3,7,5], fitness = 0.04
Fitness Selesai
[33 18 13  9 15 24]
Total Fitness : 0.3559719
Rata-rata Fitness : 0.05932865
Probabilitas : [0.08262384,0.14785318,0.2006579 ,0.28092104,0.17557566,0.11236842]
Probabilitas yang Paling Tinggi : 0.2809210419654846, pada chromosome ke 3
Chromosome yang mungkin terpilih : [1,8,2,4]


Generasi [1] ------
Chromosome 0 : [6,9,9,0], fitness = 0.045454545454545456
Chromosome 1 : [2,7,0,1], fitness = 0.09090909090909091
Chromosome 2 : [9,8,2,4], fitness = 0.05555555555555555
Chromosome 3 : [1,8,9,0], fitness = 0.06666666666666667
Chromosome 4 : [2,2,9,3], fitness = 0.0625
Chromosome 5 : [5,2,4,9], fitness 